In [61]:
from transformers import TFBertForSequenceClassification # 텍스트 분류(다대일)
from transformers import TFBertForTokenClassification # 개체명 인식(다대다)
from transformers import TFBertForQuestionAnswering # Q&A 문제(질의응답)

In [62]:
import requests

def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

train_url = 'https://korquad.github.io/dataset/KorQuAD_v1.0_train.json'
dev_url = 'https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json'

download_file(train_url, 'KorQuAD_v1.0_train.json')
download_file(dev_url, 'KorQuAD_v1.0_dev.json')

'KorQuAD_v1.0_dev.json'

In [63]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [64]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []

    for group in squad_dict['data']: # 1
        for passage in group['paragraphs']: # 1-1
            context = passage['context'] # 1-1-1 하나의 문맥 안에 
            for qa in passage['qas']: # 1-1-2 여러가지 질의응답(질문-답변) 쌍이 존재
                question = qa['question'] # 1-1-2-1 하나의 질문에
                for answer in qa['answers']: # 1-1-2-2 여러 개의 답변이 존재
                    contexts.append(context) # 문맥
                    questions.append(question) # 질문 
                    answers.append(answer) # 답변

    return contexts, questions, answers

In [65]:
train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [66]:
print('훈련 데이터의 본문 개수 : ', len(train_contexts))
print('훈련 데이터의 질문 개수 : ', len(train_questions))
print('훈련 데이터의 답변 개수 : ', len(train_answers))
print('테스트 데이터의 본문 개수 : ', len(val_contexts))
print('테스트 데이터의 질문 개수 : ', len(val_questions))
print('테스트 데이터의 답변 개수 : ', len(val_answers))

훈련 데이터의 본문 개수 :  60407
훈련 데이터의 질문 개수 :  60407
훈련 데이터의 답변 개수 :  60407
테스트 데이터의 본문 개수 :  5774
테스트 데이터의 질문 개수 :  5774
테스트 데이터의 답변 개수 :  5774


In [67]:
print(train_contexts[0])

1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [68]:
print(train_questions[0])

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [69]:
print(train_answers[0])

{'text': '교향곡', 'answer_start': 54}


In [70]:
def add_end_idx(answers, contexts): # 답변의 시작과 끝의 위치를 정확하게 기록
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip() # 오른쪽 공백 제거
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text, '골드 계산 에러'
        answer['answer_end'] = end_idx

In [71]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [72]:
print(train_answers[0])

{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [73]:
print(train_questions[1])

바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?


In [74]:
print(train_answers[1])

{'text': '1악장', 'answer_start': 421, 'answer_end': 424}


In [75]:
print(train_contexts[1])

1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [76]:
print(train_contexts[1][421])
print(train_contexts[1][422])
print(train_contexts[1][423])
print(train_contexts[1][421:424])

1
악
장
1악장


In [77]:
# 토큰화

In [78]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [79]:
def add_token_positions(encodings, answers):
    start_positions=[]
    end_positions=[]
    deleting_list=[]

    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']-1))

        if start_positions[-1] is None: # 마지막 값이 없다면
            start_positions[-1] = tokenizer.model_max_length # 모델 크기를 넣어주겠다.(사용할 수 없는 값을 넣어서 유효하지 않은 인덱스임을 나타낸다.)
            deleting_list.append(i) # 사용하지 않는 값 -> deleting_list에 넣어주겠다.

        if end_positions[-1] is None: # None이라면, 
            end_positions[-1] = tokenizer.model_max_length # 모델 길이만큼 넣어준다.(사용할 수 없는 값)
            if i not in deleting_list: # i가 deleting_list에 없다면
                deleting_list.append(i) # 삽입

    encodings.update({'start_positions':start_positions, 'end_positions':end_positions}) # 업데이트
    return deleting_list

In [80]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|███████████████████████████████████| 5774/5774 [00:00<00:00, 419648.44it/s]


In [81]:
deleting_list_for_test[:2]

[77, 583]

In [82]:
len(deleting_list_for_train), len(deleting_list_for_test), deleting_list_for_train[:5]

(267, 57, [711, 726, 728, 729, 761])

In [83]:
train_encodings[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [84]:
train_answers[0]

{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}

In [85]:
def delete_samples(encodings, deleting_list):
    input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
    attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
    start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
    end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

    X_data = [input_ids, attention_masks]
    y_data = [start_positions, end_positions]

    return X_data, y_data

In [86]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [87]:
# 모델 세팅
from transformers import TFBertModel

In [88]:
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        # Bert 모델
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 시작과 끝을 분리하기 위한 DenseLayer(시작 위치, 끝 위치)
        self.qa_outputs = tf.keras.layers.Dense(2, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), name='qa_outputs')
        # 최종 출력
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask = attention_mask) # bert를 통과

        sequence_output = outputs[0] # 첫 번째 출력을 sequence_output으로 전달받는다.

        logits = self.qa_outputs(sequence_output) # qa_outputs에 전달
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)
        # 시작 위치와 끝 위치를 저장해주고, softmax를 통과시켜줄 수 있도록 차원을 변경한다

        # softmax를 통해 시작 위치와 최종 위치를 전달
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [89]:
model = TFBertForQuestionAnswering('klue/bert-base')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [90]:
# 위 방식처럼 직접 제작해서 처리해도 괜찮고,
# 또는 transformers 내에 포함되어 있는 모델을 가져와도 상관이 없다.
from transformers import TFBertForQuestionAnswering

In [91]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer='adam', loss=loss)

In [ ]:
history = model.fit(X_train, y_train, epochs=3, verbose=1, batch_size=64)

Epoch 1/3


2024-07-24 10:25:59.010788: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/940 [..............................] - ETA: 108:31:18 - loss: 13.1384 - output_1_loss: 6.6104 - output_2_loss: 6.5280

In [97]:
# 질의응답
def predict_test_data_by_idx(idx):
    context = tokenizer.decode(X_test[0][idx]).split('[SEP]')[0]
    question = tokenizer.decode(X_test[0][idx]).split('[SEP]')[1]
    print('문맥 : ', context)
    print('질문 : ', question)
    answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
    print('정답 : ', tokenizer.decode(answer_encoded))
    output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
    start = tf.math.argmax(tf.squeeze(output[0]))
    end = tf.math.argmax(tf.squeeze(output[1]))+1
    answer_encoded = X_test[0][idx][start:end]
    print('예측 : ', tokenizer.decode(answer_encoded))
    print('-'*60)

In [98]:
for i in range(1, 10):
    predict_test_data_by_idx(i)

문맥 :  [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 :   1989년 6월 30일 평양축전에 대표로 파견 된 인물은? 
정답 :  임수경
예측 :  ##가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에
------------------------------------------------------------
문맥 :  [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 